In [1]:

%pprint
%matplotlib inline
import sys
import os.path as osp, os as os

executable_path = sys.executable
scripts_folder = osp.join(osp.dirname(executable_path), 'Scripts'); assert osp.exists(scripts_folder)
py_folder = osp.abspath(osp.join(os.pardir, 'py')); assert osp.exists(py_folder), "Create the py folder"
ffmpeg_folder = r'C:\ffmpeg\bin'; assert osp.exists(ffmpeg_folder)
shared_folder = osp.abspath(osp.join(os.pardir, 'share')); assert osp.exists(shared_folder)

if (scripts_folder not in sys.path): sys.path.insert(1, scripts_folder)
if (py_folder not in sys.path): sys.path.insert(1, py_folder)
if (ffmpeg_folder not in sys.path): sys.path.insert(1, ffmpeg_folder)
if shared_folder not in sys.path: sys.path.insert(1, shared_folder)
    
from notebook_utils import NotebookUtilities
nu = NotebookUtilities(
    data_folder_path=osp.abspath(osp.join(os.pardir, 'data')),
    saves_folder_path=osp.abspath(osp.join(os.pardir, 'saves'))
)

# Import needed libraries
import os, re
import pandas as pd
import pyperclip
import ipywidgets as widgets
from IPython.display import display

Pretty printing has been turned OFF


In [2]:

import random

def get_random_py_file(py_folder):
    """
    Walks through the specified folder and collects all .py files.
    Returns the path of a randomly selected .py file.
    
    Args:
        py_folder (str): Path to the folder to search for .py files.

    Returns:
        str: The path to a randomly selected .py file, or None if no .py files are found.
    """
    # List to store paths of .py files
    py_files = []
    
    # Walk through the folder structure
    black_list = ['.ipynb_checkpoints', '$Recycle.Bin']
    for parent_directory, _, files in os.walk(py_folder):
        if all(map(lambda x: x not in parent_directory, black_list)):
            for file in files:
                
                # Check if the file has a .py extension
                if file.endswith('.py'):
                    py_files.append(os.path.join(parent_directory, file))
    
    # If no .py files are found, return None
    if not py_files:
        return None
    
    # Randomly select and return one .py file
    return random.choice(py_files)

In [43]:

def get_random_function(verbose=True):
    function_objs_list = []
    while not function_objs_list:
        while True:
            try:
                random_py_file = get_random_py_file(py_folder)
                library_name = osp.relpath(random_py_file, py_folder).replace('.py', '')
                import_call = 'import ' + library_name
                if verbose: print(import_call)
                exec(import_call)
                break
            except (SyntaxError, ImportError, ValueError) as e:
                # nu.open_path_in_notepad(random_py_file)
                pass
        possible_attributes = [f'{library_name}.{fn}' for fn in dir(eval(library_name)) if not fn.startswith('_')]
        if verbose: print(possible_attributes)
        utils_list = [f'{library_name}.{fn}' for fn in nu.get_utility_file_functions(util_path=random_py_file) if not fn.startswith('_')]
        if verbose: print(utils_list)
        possible_functions = sorted(set(possible_attributes).intersection(set(utils_list)))
        if verbose: print(possible_functions)
        for possible_function in possible_functions:
            try:
                if verbose: print(possible_function)
                function_obj = eval(possible_function)
                if callable(function_obj):
                    function_objs_list.append(function_obj)
            except:
                continue
    random_function = random.choice(function_objs_list)
    
    return random_py_file, random_function, function_objs_list

In [55]:

import pkgutil
import inspect

standard_lib_modules = sorted([module_info.name for module_info in pkgutil.iter_modules()])
print(set(sys.builtin_module_names).intersection(set(dir(choropleth_utils))))
print(set(dir(choropleth_utils)).intersection(set(standard_lib_modules)))
for library_name in sorted(set(dir(choropleth_utils)).difference(set(standard_lib_modules)).difference(sys.builtin_module_names)):
    obj = eval(f'choropleth_utils.{library_name}')
    if isinstance(obj, type):
        print(f'choropleth_utils.{library_name} is a class.')
    elif inspect.isfunction(obj):
        print(f'choropleth_utils.{library_name} is a function.')
    else:
        print(f'choropleth_utils.{library_name} is not a class or a function.')

{'math'}
{'textwrap', 'os', 'webcolors', 'xml', 're', 'pyphen', 'pylab', 'warnings'}
choropleth_utils.ChoroplethUtilities is a class.
choropleth_utils.Element is a class.
choropleth_utils.TfidfVectorizer is a class.
choropleth_utils.__builtins__ is not a class or a function.
choropleth_utils.__cached__ is not a class or a function.
choropleth_utils.__doc__ is not a class or a function.
choropleth_utils.__file__ is not a class or a function.
choropleth_utils.__loader__ is not a class or a function.
choropleth_utils.__name__ is not a class or a function.
choropleth_utils.__package__ is not a class or a function.
choropleth_utils.__spec__ is not a class or a function.
choropleth_utils.cm is not a class or a function.
choropleth_utils.copyfile is a function.
choropleth_utils.escape is a function.
choropleth_utils.et is not a class or a function.
choropleth_utils.mpl is not a class or a function.
choropleth_utils.np is not a class or a function.
choropleth_utils.osp is not a class or a func

In [85]:

import importlib

evaluators = [fn for fn in dir(inspect) if fn.startswith('is')]
def get_dir_tree(module_name, function_calls=[]):
    try:
        module_obj = importlib.import_module(module_name)
    except:
        module_obj = eval(module_name)
    for library_name in sorted(set(dir(module_obj)).difference(set(standard_lib_modules)).difference(sys.builtin_module_names)):
        print(library_name)
        if library_name.startswith('__'):
            continue
        function_call = f'{module_name}.{library_name}'
        try:
            obj = eval(function_call)
            evaluations_list = []
            for evaluator in evaluators:
                try:
                    evaluation = eval(f'inspect.{evaluator}(obj)')
                    if evaluation:
                        evaluations_list.append(evaluator[2:])
                except:
                    continue
            if evaluations_list:
                function_calls.append(function_call)
            print(function_call, evaluations_list)
            if 'class' in evaluations_list:
                function_calls = get_dir_tree(function_call, function_calls)
        except:
            continue

        return function_calls
module_name = 'choropleth_utils'
get_dir_tree(module_name)

ChoroplethUtilities
choropleth_utils.ChoroplethUtilities ['class']
__class__
__delattr__
__dict__
__dir__
__doc__
__eq__
__format__
__ge__
__getattribute__
__getstate__
__gt__
__hash__
__init__
__init_subclass__
__le__
__lt__
__module__
__ne__
__new__
__reduce__
__reduce_ex__
__repr__
__setattr__
__sizeof__
__str__
__subclasshook__
__weakref__
add_docname
choropleth_utils.ChoroplethUtilities.add_docname ['function', 'routine']


['choropleth_utils.ChoroplethUtilities', 'choropleth_utils.ChoroplethUtilities.add_docname']

In [74]:

inspect.isclass?

Signature: inspect.isclass(object)
Docstring: Return true if the object is a class.
File:      c:\programdata\anaconda3\lib\inspect.py
Type:      function

In [65]:

f'choropleth_utils.{library_name}', f'{evaluator}(obj)'

('choropleth_utils.__builtins__', 'inspect.iskeyword(obj)')

In [44]:

random_py_file, random_function, function_objs_list = get_random_function()

import user-config
import dataframes
import tribes_layout
['tribes_layout.canvas_height', 'tribes_layout.canvas_width', 'tribes_layout.layout_df', 'tribes_layout.pd', 'tribes_layout.row_dict', 'tribes_layout.rows_list']
[]
[]
import dataframes
import dataframes
import environment
import penalty_elasticnet
import junk
[]
[]
[]
import environment
import Wordle
import penalty_elasticnet
import choropleth_utils
['choropleth_utils.ChoroplethUtilities', 'choropleth_utils.Element', 'choropleth_utils.TfidfVectorizer', 'choropleth_utils.cm', 'choropleth_utils.copyfile', 'choropleth_utils.escape', 'choropleth_utils.et', 'choropleth_utils.math', 'choropleth_utils.mpl', 'choropleth_utils.np', 'choropleth_utils.os', 'choropleth_utils.osp', 'choropleth_utils.pd', 'choropleth_utils.plt', 'choropleth_utils.pylab', 'choropleth_utils.pyphen', 'choropleth_utils.re', 'choropleth_utils.sqrt', 'choropleth_utils.textwrap', 'choropleth_utils.warnings', 'choropleth_utils.webcolors', 'choropleth_utils.xml']
['c

In [45]:

nu.describe_procedure(random_function)

In [46]:

random_function

<function simplify_segment at 0x000002A645D86AC0>

In [12]:

eval(str([f'{library_name}.{fn}' for fn in dir(library_name) if 'rel' in fn]))

['osp.relpath']


In [12]:

print([f'osp.{fn}' for fn in dir(osp) if 'rel' in fn])

['osp.relpath']


In [26]:

import glob

# Use the glob function with a recursive pattern to remove all the storage.py files
# in all the "py" folders in all the repo directories in the grandparent directory
for file_path in [osp.abspath(f) for f in glob.glob('../../*/py/storage.py', recursive=True)]:
    print(file_path)
    # os.remove(file_path)

C:\Users\daveb\OneDrive\Documents\GitHub\covid19\py\storage.py
C:\Users\daveb\OneDrive\Documents\GitHub\gpt-stuff\py\storage.py
C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\py\storage.py
C:\Users\daveb\OneDrive\Documents\GitHub\mimetic_tribes\py\storage.py
C:\Users\daveb\OneDrive\Documents\GitHub\MineCraft\py\storage.py
C:\Users\daveb\OneDrive\Documents\GitHub\notebooks\py\storage.py
C:\Users\daveb\OneDrive\Documents\GitHub\rpc\py\storage.py
C:\Users\daveb\OneDrive\Documents\GitHub\technical-interview-challenges\py\storage.py
C:\Users\daveb\OneDrive\Documents\GitHub\Wordle\py\storage.py


In [ ]:

import inspect

for cn in anova_df.columns:
    if any(map(lambda x: cn.startswith(x), ['mean_', 'sum_'])):
        function_call = 'fu.get_' + '_'.join(cn.split('_')[1:])
        try:
            nu.describe_procedure(eval(function_call))
        except Exception as e:
            print(function_call)

In [33]:

print([f'os.{fn}' for fn in dir(os) if 'list' in fn])

['os._get_exports_list', 'os.listdir']


In [35]:

# Find all the OS Path Navigation notebook files in the ipynb folders of my repos
pushables_list = nu.load_object('pushables_list')
ipynb_paths = [osp.join(file_path, 'ipynb') for file_path in pushables_list]
file_name = 'OS Path Navigation.ipynb'
file_paths_list = []
for parent_directory in ipynb_paths:
    if osp.isdir(parent_directory):
        for old_file_name in os.listdir(parent_directory):
            if old_file_name == file_name:
                old_file_path = os.path.join(parent_directory, old_file_name)
                file_paths_list.append(old_file_path)
for file_path in file_paths_list:
    print(file_path)

C:\Users\daveb\OneDrive\Documents\GitHub\covid19\ipynb\OS Path Navigation.ipynb
C:\Users\daveb\OneDrive\Documents\GitHub\itm-analysis-reporting-1\ipynb\OS Path Navigation.ipynb
C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\ipynb\OS Path Navigation.ipynb
C:\Users\daveb\OneDrive\Documents\GitHub\mimetic_tribes\ipynb\OS Path Navigation.ipynb
C:\Users\daveb\OneDrive\Documents\GitHub\notebooks\ipynb\OS Path Navigation.ipynb
C:\Users\daveb\OneDrive\Documents\GitHub\rpc\ipynb\OS Path Navigation.ipynb
C:\Users\daveb\OneDrive\Documents\GitHub\StatsByCountry\ipynb\OS Path Navigation.ipynb
C:\Users\daveb\OneDrive\Documents\GitHub\StatsByUSState\ipynb\OS Path Navigation.ipynb
C:\Users\daveb\OneDrive\Documents\GitHub\Strauss-Howe\ipynb\OS Path Navigation.ipynb
C:\Users\daveb\OneDrive\Documents\GitHub\transcriptions-notebook\ipynb\OS Path Navigation.ipynb


In [27]:

# Get the repositories directory by various means, with increasing conciseness
print([f'os.{fn}' for fn in dir(os) if ('path' in fn) and (len(f'os.{fn}') < len('osp.abspath'))])
print([f'osp.{fn}' for fn in dir(osp) if ('path' in fn) and (len(f'osp.{fn}') < len('osp.abspath'))])
print(osp.abspath(osp.join(os.curdir, os.pardir, os.pardir)))
print(osp.abspath(osp.join(nu.github_folder, os.pardir)))
print(osp.dirname(osp.dirname(osp.abspath(os.curdir))))
print(osp.dirname(nu.github_folder))

['os._fspath', 'os.defpath', 'os.fspath', 'os.path', 'os.pathsep']
[]
C:\Users\daveb\OneDrive\Documents\GitHub
C:\Users\daveb\OneDrive\Documents\GitHub
C:\Users\daveb\OneDrive\Documents\GitHub
C:\Users\daveb\OneDrive\Documents\GitHub


In [15]:

# Find all the notebook files with the word "color" in their name
black_list = ['.ipynb_checkpoints', '$Recycle.Bin']
folder_path = '../../color/ipynb'
ascii_regex = re.compile(r'[^A-Za-z0-9]+')
file_ending = '.ipynb'
file_paths_list = []
for root_dir in ['C:\\', 'D:\\']:
    for parent_directory, child_directories, files_list in os.walk(root_dir):
        if all(map(lambda x: x not in parent_directory, black_list)):
            for old_file_name in files_list:
                if old_file_name.endswith(file_ending) and ('color' in old_file_name.lower()):
                    old_file_path = os.path.join(parent_directory, old_file_name)
                    file_paths_list.append(old_file_path)

In [24]:

# Copy all the notebook files with the word "color" in their name to the color repo
import shutil

for old_file_path in file_paths_list:
    new_file_name = ascii_regex.sub('_', osp.relpath(old_file_path, osp.abspath(folder_path)).replace(file_ending, '').lstrip('.\\')) + file_ending
    new_file_path = osp.abspath(osp.join(folder_path, new_file_name))
    shutil.copy(old_file_path, new_file_path)

In [9]:

print_strs_list = ['']
print_strs_list.append('# Exclude everything except text-based OC or personal directories')
print_strs_list.append('/*')
github_folder = osp.dirname(osp.abspath(osp.curdir))
# print(github_folder)

# Get the list of all items in the parent folder and filter in only the directories
black_list = ['.ipynb_checkpoints', '$Recycle.Bin', '.git', 'data', 'saves', '__pycache__', 'LEET-CODE']
for parent_directory in os.listdir(github_folder):
    if all(map(lambda x: x not in parent_directory, black_list)) and osp.isdir(osp.join(github_folder, parent_directory)):
        print_strs_list.append(f'!/{parent_directory}')
print('\n'.join(print_strs_list))


# Exclude everything except text-based OC or personal directories
/*
!/algorithms
!/builders
!/cell_techniques
!/coin-analytics
!/curve_fitting
!/ipynb
!/json
!/library_exploration
!/load_magic
!/log
!/maintenance
!/mathematics
!/programming_languages
!/ps1
!/py
!/r_programming_language
!/statistics
!/tests
!/text_manipulation
!/visualizations
!/wheels


In [ ]:

file_path = '../data/txt/resume.txt'
nu.open_path_in_notepad(file_path)

In [ ]:

black_list = ['.ipynb_checkpoints', '$Recycle.Bin']
# walk_folder = osp.dirname(nu.github_folder)
for parent_directory, child_directories, files_list in os.walk('C:\\'):
    if all(map(lambda x: x not in parent_directory, black_list)):
        for file_name in files_list:
            if ('hair' in file_name):
                file_path = osp.join(parent_directory, file_name)
                print(file_path)

In [4]:

for parent_directory, child_directories, files_list in os.walk('C:\\'):
    for file_name in files_list:
        if file_name.endswith('.docx') and ('resume' in file_name.lower()):
            file_path = osp.join(parent_directory, file_name)
            if osp.isfile(file_path):
                print(f'{file_path}')

C:\Users\daveb\OneDrive\Documents\Babbitt Stuff\Jay_s Junk\Jay-Babbitt_Resume.docx
C:\Users\daveb\OneDrive\Documents\Google Drive\Other\Job Hunting\Dave_Babbitt_Resume (1).docx
C:\Users\daveb\OneDrive\Documents\Google Drive\Other\Job Hunting\Dave_Babbitt_Resume.docx
C:\Users\daveb\OneDrive\Documents\Google Drive\Other\Job Hunting\Dave_Babbitt_Resume_Cold_Fusion_Developer.docx
C:\Users\daveb\OneDrive\Documents\Google Drive\Other\Job Hunting\Dave_Babbitt_Resume_Delivery_Driver.docx
C:\Users\daveb\OneDrive\Documents\Google Drive\Other\Job Hunting\Dave_Babbitt_Resume_Jr._Java_Developer.docx
C:\Users\daveb\OneDrive\Documents\Google Drive\Other\Job Hunting\Dave_Babbitt_Resume_Predictive_Analyst.docx
C:\Users\daveb\OneDrive\Documents\Google Drive\Other\Job Hunting\Dave_Babbitt_Resume_QA_Tester.docx
C:\Users\daveb\OneDrive\Documents\Google Drive\Other\Job Hunting\Dave_Babbitt_Resume_Reactor_Operator.docx
C:\Users\daveb\OneDrive\Documents\Google Drive\Other\Job Hunting\Dave_Babbitt_Resume_Resea

In [13]:

# Get a list of rogue functions already in scraping utilities
scraping_utils_set = nu.get_utility_file_functions(util_path='../py/scraping_utils.py')

# Make a set of rogue su functions
fn_regex = re.compile(r'\s+"def ([a-z0-9_]+)\(')
black_list = ['.ipynb_checkpoints', '$Recycle.Bin']
rows_list = []
walk_folder = osp.dirname(osp.abspath(osp.curdir))
rogue_su_fns_set = set()
for parent_directory, child_directories, files_list in os.walk(walk_folder):
    if all(map(lambda x: x not in parent_directory, black_list)):
        for file_name in files_list:
            if file_name.endswith('.ipynb'):
                file_path = osp.join(parent_directory, file_name)
                with open(file_path, 'r', encoding='utf-8') as f:
                    lines_list = f.readlines()
                    for line in lines_list:
                        match_obj = fn_regex.search(line)
                        if match_obj:
                            fn = match_obj.group(1)
                            if fn in scraping_utils_set: rogue_su_fns_set.add(fn)
if rogue_su_fns_set:
    print(f'Search for *.ipynb; file masks in the {walk_folder} folder for this pattern:')
    print('\\s+"def (' + '|'.join(rogue_su_fns_set) + ')\(')
    print('Replace each of the calls to these definitions with calls the the su. equivalent (and delete the definitions).')

In [12]:

# Get a list of multi-notebook functions as notebook utilities candidates
fn_regex = re.compile(r'\s+"def ([a-z0-9_]+)\(')
black_list = ['.ipynb_checkpoints', '$Recycle.Bin']
rows_list = []
walk_folder = osp.dirname(osp.abspath(osp.curdir))
for parent_directory, child_directories, files_list in os.walk(walk_folder):
    if all(map(lambda x: x not in parent_directory, black_list)):
        for file_name in files_list:
            if file_name.endswith('.ipynb'):
                file_path = osp.join(parent_directory, file_name)
                with open(file_path, 'r', encoding='utf-8') as f:
                    lines_list = f.readlines()
                    fns_set = set()
                    for line in lines_list:
                        match_obj = fn_regex.search(line)
                        if match_obj:
                            fn = match_obj.group(1)
                            fns_set.add(fn)
                    for fn in fns_set:
                        row_dict = {}
                        row_dict['file_name'] = file_name
                        row_dict['function_name'] = fn
                        rows_list.append(row_dict)
function_names_df = pd.DataFrame(rows_list)
df = function_names_df.groupby('function_name').count().sort_values('file_name', ascending=False)
mask_series = (df.file_name > 1) & df.index.map(lambda x: len(x) > 4)
fns_list = df[mask_series].index.tolist()
if fns_list:
    print(f'Search for *.ipynb; file masks in the {walk_folder} folder for this pattern:')
    print('\\s+"def (' + '|'.join(fns_list) + ')\(')
    print('Find a way to get rid of these definition duplicates.')

In [7]:

import os

for parent_directory, child_directories, files_list in os.walk(r'D:\\'):
    for file_name in files_list:
        if file_name.endswith('.svg'):
            file_path = osp.join(parent_directory, file_name)
            if osp.isfile(file_path):
                with open(file_path, 'r', errors='ignore') as file:
                    data = file.read()
                    if 'Retarded' in data:
                        print(f'{file_path}')

D:\\Geek Squad Data Backup\HDD\Documents\GitHub\notebooks_old\political_triangle\svg\nick_land_comments.svg
D:\\Geek Squad Data Backup\HDD\Documents\GitHub\notebooks_old\political_triangle\svg\nick_land_comments_copy.svg
D:\\Geek Squad Data Backup\HDD\Documents\GitHub\notebooks_old\political_triangle\svg\nick_land_comments_optimized.svg
D:\\Geek Squad Data Backup\HDD\Documents\GitHub\notebooks_old\political_triangle\svg\nick_land_comments_plain.svg
D:\\Geek Squad Data Backup\HDD\Documents\GitHub\notebooks_old\political_triangle\svg\.ipynb_checkpoints\nick_land_comments_optimized-checkpoint.svg
